Import libraries requred for the script

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from numpy.polynomial import polynomial as npp
from scipy.stats import linregress

Read the CSV files

In [2]:
df_pa = pd.read_csv('VMI_Data_PA_V02_AK6_090118_032619.csv')
# df_bp1 = pd.read_csv('VMI_Data_BP1_V02_AK6.csv')
# df_bp2 = pd.read_csv('VMI_Data_BP2_V02_AK6.csv')

Check the columns in the data

In [3]:
df_pa = df_pa.drop('SPLICE_LIMIT_LOW',1)
df_pa = df_pa.drop('SPLICE_LIMIT_HIGH',1)
print(df_pa.columns)


Index(['MACHINE_ID', 'DATE_TIME_STAMP', 'GT_BARCODE', 'TIRE_TYPE',
       'CUT_LENGTH', 'CONV_WAIT_TIME', 'LFT_SPLICE_LENGTH',
       'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 'SPLICE_OK', 'PA_S8_L',
       'PA_S7_L', 'PA_S6_L', 'PA_S5_L', 'PA_S4_L', 'PA_S3_L', 'PA_S2_L',
       'PA_S1_ML', 'PA_S1_MR', 'PA_S2_R', 'PA_S3_R', 'PA_S4_R', 'PA_S5_R',
       'PA_S6_R', 'PA_S7_R', 'PA_S8_R', 'PA_PART', 'PA_BARCODE',
       'PA_BUILD_DATE', 'PA_MACHINEID'],
      dtype='object')


Sort the data by timestamp

In [4]:
df_pa.DATE_TIME_STAMP = pd.to_datetime(df_pa.DATE_TIME_STAMP, format="%m/%d/%Y %H:%M:%S.%f")


In [5]:
print(df_pa.shape)
df_pa = df_pa.drop_duplicates(subset=['CUT_LENGTH', 'CONV_WAIT_TIME', 'PA_S8_L', 'PA_S7_L', 'PA_S6_L', 'PA_S5_L', 
                                      'PA_S4_L', 'PA_S3_L', 'PA_S2_L', 'PA_S1_ML', 'PA_S1_MR', 'PA_S2_R', 
                                      'PA_S3_R', 'PA_S4_R', 'PA_S5_R', 'PA_S6_R', 'PA_S7_R', 'PA_S8_R'])
# df_pa = df_pa.drop_duplicates()
print(df_pa.shape)

(138895, 30)
(137924, 30)


Check missing data

In [6]:
df_pa['CUT_LENGTH'] = pd.to_numeric(df_pa['CUT_LENGTH'], errors='coerce')
df_pa['CONV_WAIT_TIME'] = pd.to_numeric(df_pa['CONV_WAIT_TIME'], errors='coerce')
df_pa['LFT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['LFT_SPLICE_LENGTH'], errors='coerce')
df_pa['MID_SPLICE_LENGTH'] = pd.to_numeric(df_pa['MID_SPLICE_LENGTH'], errors='coerce')
df_pa['RHT_SPLICE_LENGTH'] = pd.to_numeric(df_pa['RHT_SPLICE_LENGTH'], errors='coerce')


In [7]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))
df_pa = df_pa.dropna()



                   Total   Percent
PA_MACHINEID        1474  1.068705
PA_BARCODE          1474  1.068705
PA_PART             1474  1.068705
PA_BUILD_DATE       1474  1.068705
LFT_SPLICE_LENGTH    906  0.656884
MID_SPLICE_LENGTH    460  0.333517
RHT_SPLICE_LENGTH     31  0.022476
PA_S6_L                0  0.000000
DATE_TIME_STAMP        0  0.000000
GT_BARCODE             0  0.000000


In [8]:
total = df_pa.isnull().sum().sort_values(ascending=False)
percent = (df_pa.isnull().sum()/df_pa.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(20))



                   Total  Percent
PA_MACHINEID           0      0.0
PA_BUILD_DATE          0      0.0
DATE_TIME_STAMP        0      0.0
GT_BARCODE             0      0.0
TIRE_TYPE              0      0.0
CUT_LENGTH             0      0.0
CONV_WAIT_TIME         0      0.0
LFT_SPLICE_LENGTH      0      0.0
MID_SPLICE_LENGTH      0      0.0
RHT_SPLICE_LENGTH      0      0.0
SPLICE_OK              0      0.0
PA_S8_L                0      0.0
PA_S7_L                0      0.0
PA_S6_L                0      0.0
PA_S5_L                0      0.0
PA_S4_L                0      0.0
PA_S3_L                0      0.0
PA_S2_L                0      0.0
PA_S1_ML               0      0.0
PA_S1_MR               0      0.0


In [9]:
df_pa = df_pa.sort_values('DATE_TIME_STAMP')
df_pa.reset_index(drop=True)

,MACHINE_ID,DATE_TIME_STAMP,GT_BARCODE,TIRE_TYPE,CUT_LENGTH,CONV_WAIT_TIME,LFT_SPLICE_LENGTH,MID_SPLICE_LENGTH,RHT_SPLICE_LENGTH,SPLICE_OK,...,PA_S3_R,PA_S4_R,PA_S5_R,PA_S6_R,PA_S7_R,PA_S8_R,PA_PART,PA_BARCODE,PA_BUILD_DATE,PA_MACHINEID
0,V02,2018-09-01 00:05:28.500,AK6T27O,AK6,1709.019,71.075,11.038190,17.25513,10.414630,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
1,V02,2018-09-01 00:10:44.000,AK6T27P,AK6,1709.061,240.254,8.342554,17.77419,9.584214,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
2,V02,2018-09-01 00:13:30.900,AK6T27Q,AK6,1709.131,424.922,10.103850,19.95048,10.206630,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
3,V02,2018-09-01 00:14:31.700,AK6T27R,AK6,1708.687,5.100,13.525110,18.28922,9.897580,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
4,V02,2018-09-01 00:16:52.600,AK6T27S,AK6,1708.419,5.103,13.627740,18.70882,12.488710,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
5,V02,2018-09-01 00:17:54.500,AK6T27T,AK6,1708.447,5.090,10.933110,17.15192,10.932110,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
6,V02,2018-09-01 00:18:54.200,AK6T27U,AK6,1709.106,6.658,13.420460,17.87797,9.067024,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
7,V02,2018-09-01 00:19:52.600,AK6T27V,AK6,1708.718,5.093,12.487560,17.56590,10.518840,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
8,V02,2018-09-01 00:28:40.500,AK6T27W,AK6,1709.039,5.089,10.518410,17.77073,10.932540,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01
9,V02,2018-09-01 00:29:40.400,AK6T27X,AK6,1709.151,464.838,11.969360,17.46212,7.824645,0,...,0.05,0.05,0.05,0.05,0.0,0.0,AXLF9CA,083118041421AXLF9CA 02,14:21.5,122CL5RC01


Adding ID column for slope calculations
Probably do not need it 

In [10]:

def polyfit(x):
    return npp.polyfit(list(range(len(x))), x, 1)[0]

def linearregress(x):
    slope, intercept, r_value, p_value, std_err = linregress(list(range(len(x))), x)
    return slope


In [11]:
# This code can help write logic for roll length - it uses global var in function

# import pandas as pd
# import numpy as np

# data = np.array([[10, 2, 10, 10],
#                  [10, 3, 60, 100],
#                  [np.nan] * 4,
#                  [10, 22, 280, 250]]).T
# idx = pd.date_range('20150131', end='20150203')
# df = pd.DataFrame(data=data, columns=list('ABCD'), index=idx)
# df
#                A    B     C    D
#  =================================
#  2015-01-31    10   10    NaN  10
#  2015-02-01    2    3     NaN  22 
#  2015-02-02    10   60    NaN  280
#  2015-02-03    10   100   NaN  250

# def calculate(mul, add):
#     global value
#     value = value * mul + add
#     return value

# value = df.loc['2015-01-31', 'D']
# df.loc['2015-01-31', 'C'] = value
# df.loc['2015-02-01':, 'C'] = df.loc['2015-02-01':].apply(lambda row: calculate(*row[['A', 'B']]), axis=1)
# df
#                A    B     C     D
#  =================================
#  2015-01-31    10   10    10    10
#  2015-02-01    2    3     23    22 
#  2015-02-02    10   60    290   280
#  2015-02-03    10   100   3000  250

In [12]:
df_pa['LFT_SPLICE_DELTA'] = df_pa['LFT_SPLICE_LENGTH'] - (3.5+11.0)/2
df_pa['LFT_SPLICE_PREV'] = df_pa.LFT_SPLICE_DELTA.shift(1)
df_pa['LFT_SPLICE_MA5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['LFT_SPLICE_MA10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['LFT_SPLICE_MA20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['LFT_SPLICE_MA50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['LFT_SPLICE_SLOPE5'] = df_pa.LFT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE10'] = df_pa.LFT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE20'] = df_pa.LFT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['LFT_SPLICE_SLOPE50'] = df_pa.LFT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


df_pa['MID_SPLICE_DELTA'] = df_pa['MID_SPLICE_LENGTH'] - (4.9+18.0)/2
df_pa['MID_SPLICE_PREV'] = df_pa.MID_SPLICE_DELTA.shift(1)
df_pa['MID_SPLICE_MA5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['MID_SPLICE_MA10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['MID_SPLICE_MA20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['MID_SPLICE_MA50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['MID_SPLICE_SLOPE5'] = df_pa.MID_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE10'] = df_pa.MID_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE20'] = df_pa.MID_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['MID_SPLICE_SLOPE50'] = df_pa.MID_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)


df_pa['RHT_SPLICE_DELTA'] = df_pa['RHT_SPLICE_LENGTH'] - (3.5+11.0)/2
df_pa['RHT_SPLICE_PREV'] = df_pa.RHT_SPLICE_DELTA.shift(1)
df_pa['RHT_SPLICE_MA5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).mean()
df_pa['RHT_SPLICE_MA10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).mean()
df_pa['RHT_SPLICE_MA20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).mean()
df_pa['RHT_SPLICE_MA50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).mean()
df_pa['RHT_SPLICE_SLOPE5'] = df_pa.RHT_SPLICE_PREV.rolling(window=5,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE10'] = df_pa.RHT_SPLICE_PREV.rolling(window=10,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE20'] = df_pa.RHT_SPLICE_PREV.rolling(window=20,min_periods=1).apply(linearregress)
df_pa['RHT_SPLICE_SLOPE50'] = df_pa.RHT_SPLICE_PREV.rolling(window=50,min_periods=1).apply(linearregress)



In [13]:
# Delete first 50 rows - so that SMA and Slopes are consistent
df_pa = df_pa.iloc[50:]


In [14]:
# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             df_pa['LFT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['LFT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             df_pa['MID_SPLICE_LENGTH'].mean(), (18.0 + df_pa['MID_SPLICE_LENGTH'].mean())/2,
#             18.0,df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5,(3.5 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             df_pa['RHT_SPLICE_LENGTH'].mean(),(11.0 + df_pa['RHT_SPLICE_LENGTH'].mean())/2,
#             11.0,df_pa['RHT_SPLICE_LENGTH'].max()]

# lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#             (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

# mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, (4.9 + (18.0 - 4.9)/3),
#             (18.0 - (18.0 - 4.9)/3), 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

# rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, (3.5 + (11.0 - 3.5)/3),
#              (11.0 - (11.0 - 3.5)/3), 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

lft_bins = [df_pa['LFT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['LFT_SPLICE_LENGTH'].max()]

mid_bins = [df_pa['MID_SPLICE_LENGTH'].min(),4.9, 18.0, df_pa['MID_SPLICE_LENGTH'].max()]

rht_bins = [df_pa['RHT_SPLICE_LENGTH'].min(),3.5, 11.0, df_pa['RHT_SPLICE_LENGTH'].max()]

In [15]:
# bin_names=['Bad','OK','Good','OKH','BadH']
bin_names=['0','1','0H']

In [16]:
# def calSpliceGrade(row):
#     if row['LFT_SPLICE_GRADE'] == 'Bad' or row['MID_SPLICE_GRADE'] == 'Bad' or row['RHT_SPLICE_GRADE'] == 'Bad':
#         return 'Bad'
#     if row['LFT_SPLICE_GRADE'] == 'OK' or row['MID_SPLICE_GRADE'] == 'OK' or row['RHT_SPLICE_GRADE'] == 'OK':
#         return 'OK'
#     return 'Good'

def calSpliceGrade(row):
    if (row['LFT_SPLICE_GRADE'] == 0) or (row['MID_SPLICE_GRADE'] == 0) or (row['RHT_SPLICE_GRADE'] == 0):
        return 0
    return 1

In [17]:
# df['elderly'] = np.where(df['age']>=50, 'yes', 'no')

df_pa['LFT_SPLICE_GRADE'] = pd.cut(df_pa['LFT_SPLICE_LENGTH'],lft_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['MID_SPLICE_GRADE'] = pd.cut(df_pa['MID_SPLICE_LENGTH'],mid_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['RHT_SPLICE_GRADE'] = pd.cut(df_pa['RHT_SPLICE_LENGTH'],rht_bins, labels=bin_names, include_lowest=True).str.replace('H','').astype('int')
df_pa['SPLICE_GRADE'] = df_pa.apply(calSpliceGrade, axis=1) 


In [18]:
#df_pa['LFT_SPLICE_GRADE']
# one_hot=pd.get_dummies(df_pa['LFT_SPLICE_GRADE'])
# df_pa = df_pa.drop('LFT_SPLICE_GRADE', axis = 1)
# df_pa = df_pa.join(one_hot)


In [19]:
#df_pa.head
df_pa.to_csv('VMI_Data_PA_V02_AK6_090118_032619_mod01.csv', header=True, index=False)
